In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import collections
from collections import Counter


In [2]:
def collect_metrics(root_dir):

    depth = 2

    dict_res_roberta = dict()
    dict_res_camem = dict()
    dict_res_xlm = dict()
    
    for subdir, dirs, files in os.walk(root_dir):
        if subdir[len(root_dir):].count(os.sep) < depth:
            for file_ in files:
                if file_ == "xai_metrics.json":
                    with open(subdir + "/" + file_) as f:
                        d = json.load(f)
                        reg_log_odd = d["reg_log_odd"]
                        reg_anti_log_odd = d["reg_anti_log_odd"]
                        reg_comp = d["reg_comp"]
                        reg_suff = d["reg_suff"]
                        
                    model_type = subdir.split(os.sep)[-1].split("_")[0]
                    if model_type == "roberta":
                        dataset_name = " ".join(subdir.split(os.sep)[-1].split("_")[1:])
                        dict_res_roberta[dataset_name] = {"reg log odd": reg_log_odd, "reg anti log odd": reg_anti_log_odd, "reg comp": reg_comp, "reg suff": reg_suff}
                    elif model_type == "camem":
                        dataset_name = " ".join(subdir.split(os.sep)[-1].split("_")[1:])
                        dict_res_camem[dataset_name] = {"reg log odd": reg_log_odd, "reg anti log odd": reg_anti_log_odd, "reg comp": reg_comp, "reg suff": reg_suff}
                    elif model_type == "xlm":
                        dataset_name = " ".join(subdir.split(os.sep)[-1].split("_")[1:])
                        dict_res_xlm[dataset_name] = {"reg log odd": reg_log_odd, "reg anti log odd": reg_anti_log_odd, "reg comp": reg_comp, "reg suff": reg_suff}

    return dict_res_roberta, dict_res_camem, dict_res_xlm

def _average_dicts(dicts):
    avg_dict = collections.defaultdict(lambda: 0)
    
    for i, d in enumerate(dicts):
        for k, v in d.items():
            avg_dict[k] += v
        
    for k, v in avg_dict.items():
        avg_dict[k] /= (i+1)
    
    return avg_dict

def average_over_users(res):
    averaged_en = collections.defaultdict(list)
    averaged_it = collections.defaultdict(list)

    for k, v in res.items():
        if "en" in k and not ("nf" in k):
            averaged_en[" ".join(k.split(" ")[0:2])].append(v)
        elif "it" in k and not ("nf" in k):
            averaged_it[" ".join(k.split(" ")[0:2])].append(v)
    
    result_en = dict()
    result_it = dict()
            
    for k, v in averaged_en.items():
        result_en["avg en " + k] = _average_dicts(v)
        
    for k, v in averaged_it.items():
        result_it["avg it " + k] = _average_dicts(v)
                        
    return result_en, result_it

def process_to_present(res, column_width="1.1cm", plot_transpose=True):
    df_res = pd.DataFrame.from_dict(res).T
    # df_res = df_res.rename(columns=mapping_columns_names)
    if not plot_transpose:
        df_res = df_res.T
    s = df_res.style
    s.format(na_rep='MISS', precision=2)
    print(s.to_latex(column_format='l'+('p{'+f'{column_width}'+'}')*len(df_res.columns)))

In [3]:
dict_res_roberta, dict_res_camem, dict_res_xlm = collect_metrics("../output/complexity2")

## ROBERTA

In [4]:
avg, _ = average_over_users(dict_res_roberta)
roberta_results = {**dict_res_roberta, ** avg}

In [5]:
pd.DataFrame(roberta_results).T

,reg log odd,reg anti log odd,reg comp,reg suff
np f en6,0.254790,0.383825,0.453289,1.032911
p f en98,0.296348,0.282035,0.398688,1.022268
p f en6,0.194201,0.412143,0.344415,0.506953
p f en83,0.231219,0.332362,0.304035,0.933425
np f en57,0.071531,1.068102,0.457455,0.986791
p f en49,0.134466,0.353316,0.349116,0.696199
p f en57,0.160794,0.372913,0.310485,0.534742
np f en83,0.492532,0.553226,1.011640,1.740507
np f en49,0.077470,0.855267,0.477323,0.627773
np f en98,0.387252,0.952334,0.892267,0.851030


In [6]:
process_to_present(roberta_results)

\begin{tabular}{lp{1.1cm}p{1.1cm}p{1.1cm}p{1.1cm}}
 & reg log odd & reg anti log odd & reg comp & reg suff \\
np f en6 & 0.25 & 0.38 & 0.45 & 1.03 \\
p f en98 & 0.30 & 0.28 & 0.40 & 1.02 \\
p f en6 & 0.19 & 0.41 & 0.34 & 0.51 \\
p f en83 & 0.23 & 0.33 & 0.30 & 0.93 \\
np f en57 & 0.07 & 1.07 & 0.46 & 0.99 \\
p f en49 & 0.13 & 0.35 & 0.35 & 0.70 \\
p f en57 & 0.16 & 0.37 & 0.31 & 0.53 \\
np f en83 & 0.49 & 0.55 & 1.01 & 1.74 \\
np f en49 & 0.08 & 0.86 & 0.48 & 0.63 \\
np f en98 & 0.39 & 0.95 & 0.89 & 0.85 \\
np nf en & 0.11 & 1.18 & 0.71 & 0.77 \\
p nf en & 0.10 & 0.65 & 0.29 & 0.76 \\
avg en np f & 0.26 & 0.76 & 0.66 & 1.05 \\
avg en p f & 0.20 & 0.35 & 0.34 & 0.74 \\
\end{tabular}



## CAMEM

In [7]:
_, avg = average_over_users(dict_res_camem)
camem_results = {**dict_res_camem, ** avg}

In [8]:
pd.DataFrame(camem_results).T

,reg log odd,reg anti log odd,reg comp,reg suff
np nf it,0.300176,0.331001,0.622648,0.718047
np f it1,0.352844,0.895564,0.596366,1.680004
np f it26,0.429676,1.032176,0.715345,1.639910
np f it44,0.609396,0.343868,0.434850,0.804265
np f it43,0.245516,0.764779,0.606189,1.165382
p f it38,0.145804,0.653306,0.368455,0.704572
p f it43,0.206999,0.392977,0.408568,0.693595
p f it44,0.308696,0.488663,0.395200,0.567587
p f it26,0.269978,0.331118,0.465907,1.025204
np f it38,0.176069,0.833541,0.642736,0.539778


In [9]:
process_to_present(camem_results)

\begin{tabular}{lp{1.1cm}p{1.1cm}p{1.1cm}p{1.1cm}}
 & reg log odd & reg anti log odd & reg comp & reg suff \\
np nf it & 0.30 & 0.33 & 0.62 & 0.72 \\
np f it1 & 0.35 & 0.90 & 0.60 & 1.68 \\
np f it26 & 0.43 & 1.03 & 0.72 & 1.64 \\
np f it44 & 0.61 & 0.34 & 0.43 & 0.80 \\
np f it43 & 0.25 & 0.76 & 0.61 & 1.17 \\
p f it38 & 0.15 & 0.65 & 0.37 & 0.70 \\
p f it43 & 0.21 & 0.39 & 0.41 & 0.69 \\
p f it44 & 0.31 & 0.49 & 0.40 & 0.57 \\
p f it26 & 0.27 & 0.33 & 0.47 & 1.03 \\
np f it38 & 0.18 & 0.83 & 0.64 & 0.54 \\
p f it1 & 0.31 & 0.49 & 0.39 & 0.58 \\
p nf it & 0.21 & 0.36 & 0.35 & 0.69 \\
avg it np f & 0.36 & 0.77 & 0.60 & 1.17 \\
avg it p f & 0.25 & 0.47 & 0.41 & 0.71 \\
\end{tabular}



## XLM

In [10]:
avg_en, avg_it = average_over_users(dict_res_xlm)
xlm_results = {**dict_res_xlm, **avg_en, **avg_it}

In [11]:
pd.DataFrame(xlm_results).T

,reg log odd,reg anti log odd,reg comp,reg suff
p nf en,0.204628,0.424284,0.316101,1.094665
np f en49,0.263090,0.565619,0.609430,0.788585
np f it1,0.195089,0.721184,0.548888,1.124371
p f en83,0.228019,0.326381,0.276982,0.840931
np nf it,0.117719,0.796314,0.649633,0.619584
p f en49,0.306161,0.763289,0.287636,1.840086
np f en83,0.163308,0.811359,0.545562,0.842169
p f en98,0.182010,0.243542,0.305199,0.714898
np f en98,0.244888,0.343191,0.603611,1.387963
np f en6,0.388269,0.770802,0.866314,1.314582


In [13]:
process_to_present(xlm_results)

\begin{tabular}{lp{1.1cm}p{1.1cm}p{1.1cm}p{1.1cm}}
 & reg log odd & reg anti log odd & reg comp & reg suff \\
p nf en & 0.20 & 0.42 & 0.32 & 1.09 \\
np f en49 & 0.26 & 0.57 & 0.61 & 0.79 \\
np f it1 & 0.20 & 0.72 & 0.55 & 1.12 \\
p f en83 & 0.23 & 0.33 & 0.28 & 0.84 \\
np nf it & 0.12 & 0.80 & 0.65 & 0.62 \\
p f en49 & 0.31 & 0.76 & 0.29 & 1.84 \\
np f en83 & 0.16 & 0.81 & 0.55 & 0.84 \\
p f en98 & 0.18 & 0.24 & 0.31 & 0.71 \\
np f en98 & 0.24 & 0.34 & 0.60 & 1.39 \\
np f en6 & 0.39 & 0.77 & 0.87 & 1.31 \\
p f it38 & 0.20 & 0.30 & 0.24 & 1.28 \\
np f it38 & 0.26 & 0.35 & 0.52 & 0.95 \\
np f it26 & 0.27 & 0.74 & 0.47 & 1.70 \\
np f it44 & 0.25 & 0.99 & 0.52 & 1.69 \\
np f it43 & 0.34 & 2.00 & 0.67 & 1.87 \\
p f it1 & 0.30 & 0.40 & 0.36 & 2.04 \\
p f it26 & 0.30 & 0.30 & 0.28 & 1.06 \\
p f it44 & 0.16 & 0.22 & 0.23 & 0.94 \\
p f it43 & 0.13 & 0.17 & 0.18 & 0.81 \\
p nf it & 0.16 & 0.23 & 0.20 & 0.97 \\
np nf en & 0.18 & 0.79 & 0.66 & 0.75 \\
p f en57 & 0.22 & 0.34 & 0.23 & 0.95 \\
p f en